In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys, yaml
sys.path.insert(0,'../')
import StatPol as SP, Routines as R

basepath = os.getcwd()

AuToA = 0.5291772085**3

/home/marco/Data/RICERCA/DFT AND MANY BODY/LR MOLECULES/Statical polarizability/Analysis/Calculations


# Data analysis

In [2]:
nsp_results = yaml.load(open('nsp_results.yaml'))
#nsp_results

In [3]:
def build_computed_mol_list(dataset):
    """
    Build the list of molecules for which the 'results' key is provided     
    """
    computed_mol = []
    for mol,data in nsp_results.iteritems():
        if 'results' in data:
            computed_mol.append(mol)
    computed_mol.sort()
    return computed_mol

In [4]:
computed_mol = build_computed_mol_list(nsp_results)
print 'Number of computed molecules', len(computed_mol)

Number of computed molecules 75


## First checks of the convergence procedure

We check if convergence has been achieved, both on the field intensity and on the value of rmult

In [5]:
for mol in computed_mol:
    results = nsp_results[mol]['results']
    for study in results:
        if results[study]['field_conv']['converged']==False:
            print mol, study, 'field convergence failed'
        if results[study]['rmult_conv']['converged']==False:
            print mol, study, 'rmult convergence failed'

NaCl ('lda_pw', 'hgh_k') field convergence failed


The field convergence on this study has to be further investigated

For each study we extract the convergend values of the field or the value of rmult

In [7]:
print 'mol', 'study', 'field_conv', 'rmult_conv'
for mol in computed_mol:
    print mol
    results = nsp_results[mol]['results']
    for study in results:
        print ' ', study,results[study]['field_conv']['converged_value'],\
        results[study]['rmult_conv']['converged_value']

mol study field_conv rmult_conv
AlF
  ('lda_pw', 'hgh_k') 0.01 6.0
  ('pbe', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_ss') 0.01 6.0
  ('lda_pt', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_aw') 0.01 6.0
Ar
  ('lda_pw', 'hgh_k') 0.01 7.0
  ('pbe', 'hgh_k') 0.01 7.0
  ('lda_pt', 'hgh_k') 0.01 7.0
BF
  ('lda_pw', 'hgh_k') 0.01 6.0
  ('pbe', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_ss') 0.01 6.0
  ('lda_pt', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_aw') 0.01 6.0
BH2Cl
  ('lda_pw', 'hgh_k') 0.01 6.0
  ('pbe', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_ss') 0.01 6.0
  ('lda_pt', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_aw') 0.01 6.0
BH2F
  ('lda_pw', 'hgh_k') 0.01 6.0
  ('pbe', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_ss') 0.01 6.0
  ('lda_pt', 'hgh_k') 0.01 6.0
  ('pbe', 'nlcc_aw') 0.01 6.0
BH3
  ('lda_pw', 'hgh_k') 0.01 5.0
  ('pbe', 'hgh_k') 0.01 5.0
  ('pbe', 'nlcc_ss') 0.01 5.0
  ('lda_pt', 'hgh_k') 0.01 5.0
  ('pbe', 'nlcc_aw') 0.01 5.0
BHF2
  ('lda_pw', 'hgh_k') 0.01 5.0
  ('pbe', 'hgh_k') 0.01 5.0
  ('pbe', 'nlcc_ss') 0.01 5.0
  ('lda_p

We see that both the field convergence and the rmult convergence values changes for the different molecules and studies of the dataset. This fact indicates that the (quite complex) procedure for the determination of these parameters has been useful.

## RMSRE and MRE for each study of the nsp dataset

We compute the RMSRE and MRE for each study and collect the result in a table. This analysis provides a measurement of the effects of the psp on the determination of the statical polarizability for the molecules of the dataset.

In [8]:
def get_study_alpha_diag(data,xc,psp):
    """
    Extract the diagonal part of the polarizability associated to the study xc,psp taking the
    dictionary data as input. If xcx,psp is not in the computed study returns None.
    
    Args:
        data : is of the form nsp_results[molecule]['results']
    """
    study = data.keys()
    alpha = None
    if (xc,psp) in study:
        results = data[xc,psp]['rmult_conv']
        rmult = results['converged_value']
        alpha = results['results'][rmult]
        alpha = np.diag(alpha) 
    return alpha

def eval_relative_error(dataset,molecule,xc,psp):
    """"
    Compute the relative error associated to the selected molecule in given
    study. Return an array built as 
    eps_i = (alpha_ii -alpha_ref_ii)/alpha_ref_ii * 100
    If the reference results for the selected xc and/or the the results for
    the given study (xc+psp) are absent it returns None.
    """
    eps = None
    ref_data = dataset[molecule]['ref_results'] 
    data = dataset[molecule]['results']
    if xc in ref_data and (xc,psp) in data :
        alpha_ii = get_study_alpha_diag(data,xc,psp)
        alpha_ref_ii = np.array(ref_data[xc])/AuToA
        eps = 100.0 * (alpha_ii-alpha_ref_ii)/alpha_ref_ii
    
    return eps

def eval_sre_molecule(dataset,mol,xc,psp):
    """
    Compute the (averageed) squared relative error for a single molecule, associated
    to the study xc+psp. 
    """
    sre = None
    eps = eval_relative_error(dataset,mol,xc,psp)
    if not (eps is None):
        sre = sum(eps**2)/3.  
    return sre

def eval_re_molecule(dataset,mol,xc,psp):
    """
    Compute the (average) relative error for a single molecule, associated
    to the study xc+psp. 
    """
    re = None
    eps = eval_relative_error(dataset,mol,xc,psp)
    if not (eps is None):
        re = sum(eps)/3.  
    return re

def eval_rmsre(dataset,xc,psp):
    """
    Compute the RMSRE, as defined in the paper of HG, associated
    to the study xc+psp. 
    """
    rmsre = 0.
    N = 0 
    for mol in build_computed_mol_list(dataset):
        sre = eval_sre_molecule(dataset,mol,xc,psp)
        if not (sre is None):
            rmsre+=sre
            N+=1
    if N>0 : rmsre = rmsre/N
    return np.sqrt(rmsre)

def eval_mre(dataset,xc,psp):
    """
    Compute the MRE, as defined in the paper of HG, associated
    to the study xc+psp. 
    """
    mre = 0.
    N = 0 
    for mol in build_computed_mol_list(dataset):
        re = eval_re_molecule(dataset,mol,xc,psp)
        if not (re is None):
            mre+=re
            N+=1
    if N>0 : mre = mre/N
    return mre

def eval_rmsre_old(dataset,xc,psp):
    """
    Compute the RMSRE, as defined in the paper of HG, associated
    to the study xc+psp. 
    """
    rmsre = 0.
    N = 0 
    for mol in build_computed_mol_list(dataset):
        eps = eval_relative_error(dataset,mol,xc,psp)
        #print mol,eps
        if not (eps is None):
            rmsre += sum(eps**2)/3.  
            N += 1
    #print 'number of contributions to rmsre', N
    if N>0 : rmsre = rmsre/N
    
    return np.sqrt(rmsre)
 
def eval_mre_old(dataset,xc,psp):
    """
    Compute the MRE, as defined in the paper of HG, associated
    to the study xc+psp. 
    """
    mre = 0.
    N = 0 
    for mol in build_computed_mol_list(dataset):
        eps = eval_relative_error(dataset,mol,xc,psp)
        if not (eps is None):
            mre += sum(eps)/3.  
            N += 1
    #print 'number of contributions to mre', N
    if N>0 : mre = mre/N
    
    return mre

Compute the RMSRE and MRE errors for all the performed study

In [9]:
reduced_study_set = [('lda_pt','hgh_k'),('lda_pw','hgh_k'),('pbe','hgh_k'),\
                     ('pbe','nlcc_aw'),('pbe','nlcc_ss')] #('pbe0','hgh_k')

In [10]:
print 'study', 'RMSRE', 'MRE'
for study in reduced_study_set:
    print '{} {:1.2f} {:1.2f}'.format(study,eval_rmsre(nsp_results,study[0],study[1]),\
                                 eval_mre(nsp_results,study[0],study[1]))

study RMSRE MRE
('lda_pt', 'hgh_k') 0.00 0.00
('lda_pw', 'hgh_k') 3.05 0.11
('pbe', 'hgh_k') 3.06 0.17
('pbe', 'nlcc_aw') 3.29 0.16
('pbe', 'nlcc_ss') 3.42 0.08


Compute the array with the relative error for each molecule and study

In [28]:
print 'molecule', 'study', 'Relative error'
for mol in computed_mol:
    print mol
    for study in reduced_study_set:
        re = eval_relative_error(nsp_results,mol,study[0],study[1])
        re_str = ''
        if not (re is None):
            re_str = ['{:1.2f}'.format(x) for x in re]
        print ' ',study,re_str

molecule study Relative error
AlF
  ('lda_pt', 'hgh_k') 
  ('lda_pw', 'hgh_k') ['0.15', '0.15', '0.19']
  ('pbe', 'hgh_k') ['-0.59', '-0.59', '-0.04']
  ('pbe', 'nlcc_aw') ['1.00', '1.00', '0.75']
  ('pbe', 'nlcc_ss') ['-0.35', '-0.35', '0.11']
Ar
  ('lda_pt', 'hgh_k') 
  ('lda_pw', 'hgh_k') ['-0.35', '-0.35', '-0.35']
  ('pbe', 'hgh_k') ['-0.31', '-0.31', '-0.31']
  ('pbe', 'nlcc_aw') 
  ('pbe', 'nlcc_ss') 
BF
  ('lda_pt', 'hgh_k') 
  ('lda_pw', 'hgh_k') ['0.57', '0.57', '0.37']
  ('pbe', 'hgh_k') ['0.39', '0.39', '0.14']
  ('pbe', 'nlcc_aw') ['0.54', '0.54', '0.47']
  ('pbe', 'nlcc_ss') ['0.39', '0.39', '1.04']
BH2Cl
  ('lda_pt', 'hgh_k') 
  ('lda_pw', 'hgh_k') ['0.26', '-0.06', '0.04']
  ('pbe', 'hgh_k') ['0.29', '-0.07', '-0.01']
  ('pbe', 'nlcc_aw') ['-0.19', '-0.53', '-0.57']
  ('pbe', 'nlcc_ss') ['-0.29', '-0.71', '-0.56']
BH2F
  ('lda_pt', 'hgh_k') 
  ('lda_pw', 'hgh_k') ['0.06', '-0.43', '-0.00']
  ('pbe', 'hgh_k') ['0.06', '-0.45', '0.03']
  ('pbe', 'nlcc_aw') ['0.00', '-0.50

In [11]:
print 'molecule', 'study', 'RMSRE', 'MRE'
for mol in computed_mol:
    print mol
    for study in reduced_study_set:
        #print ' ',study,eval_relative_error(nsp_results,mol,study[0],study[1])
        sre = eval_sre_molecule(nsp_results,mol,study[0],study[1])
        rsre = ''
        if not (sre is None):
            rsre = '{:1.2f}'.format(np.sqrt(sre))
        re = eval_re_molecule(nsp_results,mol,study[0],study[1])
        re_string = ''
        if not (re is None):
            re_string = '{:1.2f}'.format(re)
        print '  {} {} {}'.format(study,rsre,re_string)

molecule study RMSRE MRE
AlF
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.16 0.16
  ('pbe', 'hgh_k') 0.48 -0.41
  ('pbe', 'nlcc_aw') 0.93 0.92
  ('pbe', 'nlcc_ss') 0.29 -0.20
Ar
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.35 -0.35
  ('pbe', 'hgh_k') 0.31 -0.31
  ('pbe', 'nlcc_aw')  
  ('pbe', 'nlcc_ss')  
BF
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.51 0.50
  ('pbe', 'hgh_k') 0.33 0.31
  ('pbe', 'nlcc_aw') 0.52 0.52
  ('pbe', 'nlcc_ss') 0.68 0.61
BH2Cl
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.15 0.08
  ('pbe', 'hgh_k') 0.17 0.07
  ('pbe', 'nlcc_aw') 0.46 -0.43
  ('pbe', 'nlcc_ss') 0.55 -0.52
BH2F
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.25 -0.12
  ('pbe', 'hgh_k') 0.26 -0.12
  ('pbe', 'nlcc_aw') 0.30 -0.22
  ('pbe', 'nlcc_ss') 0.46 -0.36
BH3
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.82 -0.78
  ('pbe', 'hgh_k') 0.90 -0.83
  ('pbe', 'nlcc_aw') 0.88 -0.84
  ('pbe', 'nlcc_ss') 1.09 -1.06
BHF2
  ('lda_pt', 'hgh_k')  
  ('lda_pw', 'hgh_k') 0.41 -0.34
  ('pbe'

It emerges that the relative error is about 25% for H. Probably the H "molecule" has to be treated within the sp framework.

Compute again RMSRE and MRE purging H from the results

In [30]:
from copy import deepcopy
nsp_results_popH = deepcopy(nsp_results)
nsp_results_popH.pop('H',None)

print 'study', 'RMSRE', 'MRE', '(without H)'
for study in reduced_study_set:
    print '{} {:1.2f} {:1.2f}'.format(study,eval_rmsre(nsp_results_popH,study[0],study[1]),\
                                 eval_mre(nsp_results_popH,study[0],study[1]))

study RMSRE MRE (without H)


KeyError: 'H'